In [ ]:
!pip install unidecode
!pip install emot --upgrade
!pip install emoji
!pip install autocorrect
!!pip install contractions
!pip install wordcloud

import pandas as pd
import unidecode
import re
from emot.emo_unicode import UNICODE_EMOJI,EMOTICONS_EMO
import nltk
import emoji
from nltk.tokenize import word_tokenize 
from nltk.stem import WordNetLemmatizer 
from autocorrect import Speller 
import contractions
from nltk.corpus import stopwords


import seaborn as sb
import numpy as np
from collections import  Counter
from sklearn.feature_extraction.text import CountVectorizer
from wordcloud import WordCloud, STOPWORDS
import matplotlib.pyplot as plt 
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

In [4]:
# Load data from CSV file
data = pd.read_csv("/Users/wenxiu/Documents/GitHub/CZ4034-Information-Retrieval-Grp-28/data/labelled/combined_labelled.csv", encoding="ISO-8859-1")
data

,username,verified,followersCount,rawContent,date,replyCount,retweetCount,likeCount,url,hashtags,game,labels
0,G_F758,FALSE,343,Hogwarts Legacy\r\nFlight the Good Flight (SIL...,18/2/2023 12:47,0,0,1.0,https://twitter.com/G_F758/status/162692648616...,"['PlayStationTrophy', 'PS5Share', 'HogwartsLeg...",HGL,0.0
1,gjvarkey,FALSE,44,Add leads from Facebook Lead Ads to a Google S...,13/2/2023 7:17,0,0,0.0,https://twitter.com/gjvarkey/status/1625031302...,"['cats', 'happydinoday', 'shopping', 'aiartcom...",HGL,0.0
2,DeVinSimmons,FALSE,421,Oh and how smooth getting on and off the broom...,16/2/2023 5:27,0,0,0.0,https://twitter.com/DeVinSimmons/status/162609...,['HogwartsLegacy'],HGL,0.0
3,SirenWolfxx,FALSE,122,It's finally here! ð­ð¥¹ @twitchSIE #hp #ho...,13/2/2023 20:11,0,0,2.0,https://twitter.com/SirenWolfxx/status/1625226...,"['hp', 'hogwartlegacy', 'slytherin']",HGL,0.0
4,cozzboz,FALSE,117,Playing endless hours of #HogwartsLegacy got m...,16/2/2023 14:34,0,0,0.0,https://twitter.com/cozzboz/status/16262284309...,['HogwartsLegacy'],HGL,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
94560,Abfun77,FALSE,382,Video of ELDEN RING: All Quests in Order + Mis...,22/2/2023 0:17,0,7,79.0,https://twitter.com/Abfun77/status/16281871857...,"['ELDENRING', 'ELDEN_RING', 'guide', 'gaming',...",ELR,NaN
94561,Vivern96,FALSE,7,Red Wolf of Radagon RL1 NoHit #ELDEN_RING #ELD...,16/6/2022 14:29,0,0,1.0,https://twitter.com/Vivern96/status/1537442134...,"['ELDEN_RING', 'ELDENRING', 'gaming', 'game']",ELR,NaN
94562,EllieCupcakes,FALSE,263,Fanart of Miruku with her Apostle (Slim) and M...,3/3/2023 21:01,1,7,32.0,https://twitter.com/EllieCupcakes/status/16317...,"['EldenRing', 'EldenRingArt']",ELR,NaN
94563,Pinknotess,FALSE,0,This comic is definitely worth reading! Come c...,19/3/2023 3:42,0,0,0.0,https://twitter.com/Pinknotess/status/16372984...,"['EldenRing', 'illusrtationart', 'Romance']",ELR,NaN


In [5]:
# Remove duplicate tweets
data = data.drop_duplicates(subset=["rawContent"])
data

,username,verified,followersCount,rawContent,date,replyCount,retweetCount,likeCount,url,hashtags,game,labels
0,G_F758,FALSE,343,Hogwarts Legacy\r\nFlight the Good Flight (SIL...,18/2/2023 12:47,0,0,1.0,https://twitter.com/G_F758/status/162692648616...,"['PlayStationTrophy', 'PS5Share', 'HogwartsLeg...",HGL,0.0
1,gjvarkey,FALSE,44,Add leads from Facebook Lead Ads to a Google S...,13/2/2023 7:17,0,0,0.0,https://twitter.com/gjvarkey/status/1625031302...,"['cats', 'happydinoday', 'shopping', 'aiartcom...",HGL,0.0
2,DeVinSimmons,FALSE,421,Oh and how smooth getting on and off the broom...,16/2/2023 5:27,0,0,0.0,https://twitter.com/DeVinSimmons/status/162609...,['HogwartsLegacy'],HGL,0.0
3,SirenWolfxx,FALSE,122,It's finally here! ð­ð¥¹ @twitchSIE #hp #ho...,13/2/2023 20:11,0,0,2.0,https://twitter.com/SirenWolfxx/status/1625226...,"['hp', 'hogwartlegacy', 'slytherin']",HGL,0.0
4,cozzboz,FALSE,117,Playing endless hours of #HogwartsLegacy got m...,16/2/2023 14:34,0,0,0.0,https://twitter.com/cozzboz/status/16262284309...,['HogwartsLegacy'],HGL,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
94560,Abfun77,FALSE,382,Video of ELDEN RING: All Quests in Order + Mis...,22/2/2023 0:17,0,7,79.0,https://twitter.com/Abfun77/status/16281871857...,"['ELDENRING', 'ELDEN_RING', 'guide', 'gaming',...",ELR,NaN
94561,Vivern96,FALSE,7,Red Wolf of Radagon RL1 NoHit #ELDEN_RING #ELD...,16/6/2022 14:29,0,0,1.0,https://twitter.com/Vivern96/status/1537442134...,"['ELDEN_RING', 'ELDENRING', 'gaming', 'game']",ELR,NaN
94562,EllieCupcakes,FALSE,263,Fanart of Miruku with her Apostle (Slim) and M...,3/3/2023 21:01,1,7,32.0,https://twitter.com/EllieCupcakes/status/16317...,"['EldenRing', 'EldenRingArt']",ELR,NaN
94563,Pinknotess,FALSE,0,This comic is definitely worth reading! Come c...,19/3/2023 3:42,0,0,0.0,https://twitter.com/Pinknotess/status/16372984...,"['EldenRing', 'illusrtationart', 'Romance']",ELR,NaN


In [6]:
print("Shape of data =>",data.shape)

Shape of data => (94555, 12)


In [7]:
# Check for null tweets
data['rawContent'].isnull().sum()

0

In [8]:
data['rawContent']= [" ".join(x.split()) for x in data['rawContent']]

/var/folders/n8/3f_jyqrd5r97gbpz42mt5mn00000gn/T/ipykernel_19193/4120841830.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['rawContent']= [" ".join(x.split()) for x in data['rawContent']]


In [9]:
def accented_characters_removal(text):
    # this is a docstring
    """
    The function will remove accented characters from the 
    text contained within the Dataset.
       
    arguments:
        input_text: "text" of type "String". 
                    
    return:
        value: "text" with removed accented characters.
        
    Example:
    Input : Málaga, àéêöhello
    Output : Malaga, aeeohello    
        
    """
    # Remove accented characters from text using unidecode.
    # Unidecode() - It takes unicode data & tries to represent it to ASCII characters. 
    text = unidecode.unidecode(text)
    return text

In [10]:
def tweet_to_words(tweet):
    text = tweet.lower()
   
    # Code to remove the Hashtags from the text
    text=re.sub(r'\B#\S+','',text)

    # print(text)
    # Code to remove the links from the text
    text=re.sub(r"http\S+", "",text)
    text=re.sub(r"\ [A-Za-z]*\.com", " ",text)
    # Code to substitute the multiple spaces with single spaces
    text=re.sub(r'\s+', ' ', text, flags=re.I)
    # Remove the twitter handlers
    text=re.sub('@[^\s]+','',text)

    # Code to remove the Special characters from the text 
    # text=' '.join(re.findall(r'\w+', text))

    return text

In [11]:
print(tweet_to_words(data.iloc[1]['rawContent']))

add leads from facebook lead ads to a google sheets spreadsheet ðautomation template â ð 


In [12]:
spamData= data.loc[data['rawContent'].str.contains("live stream|live|stream|automation|template|twitch|podcast", case=False)]
spamData['rawContent']

1        Add leads from Facebook Lead Ads to a Google S...
3        It's finally here! ð­ð¥¹ @twitchSIE #hp #ho...
13       I have some errands today, but I should be liv...
18       Going live with #HogwartLegacy! Get your #twit...
20       ð¥µð¥µ Sebastian and Ominis over here having...
                               ...                        
94544    #goinglive in about 30 minutes #ms__metal on #...
94545    Alright its time to #Lurk #smallstreamers. Her...
94546    Shall we continue our @ELDENRING journey Ladie...
94554    Elden Ring has a jump scare or 2, so be sure t...
94556    Yo! The other half has started to stream! He's...
Name: rawContent, Length: 15005, dtype: object

In [13]:
data = data[data["rawContent"].str.contains("live stream|live|stream|automation|template|twitch|podcast", 
    case=False)==False]
data

,username,verified,followersCount,rawContent,date,replyCount,retweetCount,likeCount,url,hashtags,game,labels
0,G_F758,FALSE,343,Hogwarts Legacy Flight the Good Flight (SILVER...,18/2/2023 12:47,0,0,1.0,https://twitter.com/G_F758/status/162692648616...,"['PlayStationTrophy', 'PS5Share', 'HogwartsLeg...",HGL,0.0
2,DeVinSimmons,FALSE,421,Oh and how smooth getting on and off the broom...,16/2/2023 5:27,0,0,0.0,https://twitter.com/DeVinSimmons/status/162609...,['HogwartsLegacy'],HGL,0.0
4,cozzboz,FALSE,117,Playing endless hours of #HogwartsLegacy got m...,16/2/2023 14:34,0,0,0.0,https://twitter.com/cozzboz/status/16262284309...,['HogwartsLegacy'],HGL,0.0
5,Fantastic_4ork,FALSE,486,Hey #AvalancheSoftware how about that #photomo...,15/2/2023 19:24,0,0,3.0,https://twitter.com/Fantastic_4ork/status/1625...,"['AvalancheSoftware', 'photomode', 'hogwartsle...",HGL,0.0
6,jordan_sebas2,FALSE,487,Hogwarts Legacy Flight the Good Flight (SILVER...,15/2/2023 19:57,0,0,0.0,https://twitter.com/jordan_sebas2/status/16259...,"['PlayStationTrophy', 'PS5Share', 'HogwartsLeg...",HGL,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
94560,Abfun77,FALSE,382,Video of ELDEN RING: All Quests in Order + Mis...,22/2/2023 0:17,0,7,79.0,https://twitter.com/Abfun77/status/16281871857...,"['ELDENRING', 'ELDEN_RING', 'guide', 'gaming',...",ELR,NaN
94561,Vivern96,FALSE,7,Red Wolf of Radagon RL1 NoHit #ELDEN_RING #ELD...,16/6/2022 14:29,0,0,1.0,https://twitter.com/Vivern96/status/1537442134...,"['ELDEN_RING', 'ELDENRING', 'gaming', 'game']",ELR,NaN
94562,EllieCupcakes,FALSE,263,Fanart of Miruku with her Apostle (Slim) and M...,3/3/2023 21:01,1,7,32.0,https://twitter.com/EllieCupcakes/status/16317...,"['EldenRing', 'EldenRingArt']",ELR,NaN
94563,Pinknotess,FALSE,0,This comic is definitely worth reading! Come c...,19/3/2023 3:42,0,0,0.0,https://twitter.com/Pinknotess/status/16372984...,"['EldenRing', 'illusrtationart', 'Romance']",ELR,NaN


In [14]:
# Converting emojis to words
def convert_emojis(text):
    for emot in UNICODE_EMOJI:
        text = text.replace(emot, "_".join(UNICODE_EMOJI[emot].replace(",","").replace(":","").split()))
        return text

# Converting emoticons to words    
def convert_emoticons(text):
    for emot in EMOTICONS_EMO:
        text = re.sub(u'\('+emot+'\)', "_".join(EMOTICONS_EMO[emot].replace(",","").split()), text)
        return text

#for now replaced emojis and emoticons instead of removing them
def remove_emoticons(text):
    emoticon_pattern = re.compile(u'(' + u'|'.join(k for k in EMOTICONS) + u')')
    return emoticon_pattern.sub(r'', text)

def load_dict_smileys():
    
    return {
        ":‑)":"smiley",
        ":-]":"smiley",
        ":-3":"smiley",
        ":->":"smiley",
        "8-)":"smiley",
        ":-}":"smiley",
        ":)":"smiley",
        ":]":"smiley",
        ":3":"smiley",
        ":>":"smiley",
        "8)":"smiley",
        ":}":"smiley",
        ":o)":"smiley",
        ":c)":"smiley",
        ":^)":"smiley",
        "=]":"smiley",
        "=)":"smiley",
        ":-))":"smiley",
        ":‑D":"smiley",
        "8‑D":"smiley",
        "x‑D":"smiley",
        "X‑D":"smiley",
        ":D":"smiley",
        "8D":"smiley",
        "xD":"smiley",
        "XD":"smiley",
        ":‑(":"sad",
        ":‑c":"sad",
        ":‑<":"sad",
        ":‑[":"sad",
        ":(":"sad",
        ":c":"sad",
        ":<":"sad",
        ":[":"sad",
        ":-||":"sad",
        ">:[":"sad",
        ":{":"sad",
        ":@":"sad",
        ">:(":"sad",
        ":'‑(":"sad",
        ":'(":"sad",
        ":‑P":"playful",
        "X‑P":"playful",
        "x‑p":"playful",
        ":‑p":"playful",
        ":‑Þ":"playful",
        ":‑þ":"playful",
        ":‑b":"playful",
        ":P":"playful",
        "XP":"playful",
        "xp":"playful",
        ":p":"playful",
        ":Þ":"playful",
        ":þ":"playful",
        ":b":"playful",
        "<3":"love"
        }

def remove_special_characters(text):
  return re.sub("[^A-z0-9]", " ", text) #so it allows only alphabet characters and numerical characters.

def remove_links(text):
  return re.sub("[-a-zA-Z0-9@:%._\+~#=]{1,256}\.[a-zA-Z0-9()]{1,6}\b([-a-zA-Z0-9()@:%_\+.~#?&//=]*)", " ", text) #got the regex for matching links from stackoverflow

#both hashtags and handles can contain alphanumeric characters and underscores only

def remove_hashtags(text):
  return re.sub("\B\#[A-z0-9_]+\\b", "", text) #removes the entire hashtag
  #return re.sub("\#", "", text) #for removing just the '#' but leaving the word following

def remove_twitter_handles(text):
  return re.sub("\B\@[A-z0-9_]+\\b", "", text) #removes twitter handles

# The code for spelling corrections
def spelling_correction(text):
    ''' 
    This function will correct spellings.
    
    arguments:
         input_text: "text" of type "String".
         
    return:
        value: Text after corrected spellings.
        
    Example: 
    Input : This is Oberois from Dlhi who came heree to studdy.
    Output : This is Oberoi from Delhi who came here to study.
      
    
    '''
    # Check for spellings in English language
    spell = Speller(lang='en')
    Corrected_text = spell(text)
    return Corrected_text

def clean_ascii(text):
  # function to remove non-ASCII chars from data
  return ''.join(i for i in text if ord(i) < 128)

def expand_contractions(text):
  return contractions.fix(text)

w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
lemmatizer = nltk.stem.WordNetLemmatizer()
def lemmatization(text):
    """This function converts word to their root words 
       without explicitely cut down as done in stemming.
    
    arguments:
         input_text: "text" of type "String".
         
    return:
        value: Text having root words only, no tense form, no plural forms
        
    Example: 
    Input : text reduced 
    Output :  text reduce
    
   """
    # Converting words to their root forms
    
    lemma = [lemmatizer.lemmatize(w,'v') for w in w_tokenizer.tokenize(text)]
    return lemma

def tweet_to_words(tweet):

    text = tweet.lower()
   
    # both hashtags and handles can contain alphanumeric characters and underscores only  
    # Code to remove the Hashtags from the text
    text = re.sub("\B\#[A-z0-9_]+\\b", "", text) #corrected regex

    # Code to remove the links from the text
    text = re.sub("[-a-zA-Z0-9@:%._\+~#=]{1,256}\.[a-zA-Z0-9()]{1,6}\b([-a-zA-Z0-9()@:%_\+.~#?&//=]*)", " ", text) #corrected regex

    # Code to substitute the multiple spaces with single spaces
    text=re.sub(r'\s+', ' ', text, flags=re.I)
    # Remove the twitter handlers
    text = re.sub("\B\@[A-z0-9_]+\\b", "", text) # corrected regex
    # print(text)

    # text=convert_emoticons(text)

    SMILEY = load_dict_smileys()  
    words = text.split()
    reformed = [SMILEY[word] if word in SMILEY else word for word in words]
    text = " ".join(reformed)

    # text=convert_emojis(text)
    #can check which version is better
    text = emoji.demojize(text)
    # print(text)
    text=accented_characters_removal(text)
    text=expand_contractions(text)

    # Code to remove the Special characters from the text 
    text=' '.join(re.findall(r'\w+', text))
    
    text=spelling_correction(text)
    text=lemmatization(text)
    # text=clean_ascii(text)


    return text

    # tokenize
    # words = text.split()

In [15]:
cleantext=[]
i=0
for item in data['rawContent']:
    if type(item)==float:
      item = str(item)
    words=tweet_to_words(item)
    cleantext+=[words]
    i+=1
    if i%100==0:
      print(i)
data['processed']=cleantext
data

100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900


KeyboardInterrupt: 

In [43]:
data.to_csv('combined_labelled_processed.csv',index = False)